<!--
#  Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
#    Licensed under the Apache License, Version 2.0 (the "License").
#    You may not use this file except in compliance with the License.
#    You may obtain a copy of the License at
#
#        http://www.apache.org/licenses/LICENSE-2.0
#
#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.
-->

# This notebook creates an image with additional Applications that can be used alongside Jupyter

## Content
1. [Configuration](#Configuration)
2. [Build Image](#Build-Image)


### Configuration

In [1]:
image_name = 'jupyter-with-apps'
folder_name = '/efs/shared/samples/notebooks/M-Admin/extra_user_apps/'

In [2]:
%cd $folder_name

/efs/shared/samples/notebooks/M-Admin/extra_user_apps


### Build Image

Lets see how  orbit build image works...

get our orbit env and team names

In [3]:
env_name = %env AWS_ORBIT_ENV
team_name = %env AWS_ORBIT_TEAM_SPACE
(env_name,team_name)

('test-env-public', 'lake-creator')

Repository name will be created from the image name prefixed by the env context.  Users are only able to manipulate ECR repos that start with 'orbit-{env_name}-users-'

In [4]:
repository_name = (f"orbit-{env_name}-users-{image_name}")
repository_name

'orbit-test-env-public-users-jupyter-with-apps'

In [5]:
!aws ecr delete-repository --repository-name $repository_name --force


An error occurred (RepositoryNotFoundException) when calling the DeleteRepository operation: The repository with name 'orbit-test-env-public-users-jupyter-with-apps' does not exist in the registry with id '495869084367'


adding a file to our docker as an example

In [6]:
pwd = %pwd
pwd

'/efs/shared/samples/notebooks/M-Admin/extra_user_apps'

Now lets run the command

In [7]:
%%time

output = !orbit build image -e $env_name -d $pwd -n $image_name 
output

CPU times: user 23.4 ms, sys: 10.7 ms, total: 34.1 ms
Wall time: 6min 54s


['[2021-04-01 20:31:58,452][__main__.py  :368] env: test-env-public',
 '[2021-04-01 20:31:58,452][__main__.py  :369] dir: /efs/shared/samples/notebooks/M-Admin/extra_user_apps',
 '[2021-04-01 20:31:58,452][__main__.py  :370] name: jupyter-with-apps',
 '[2021-04-01 20:31:58,452][__main__.py  :371] script: None',
 '[2021-04-01 20:31:58,452][__main__.py  :372] timeout: 30',
 '[2021-04-01 20:31:58,452][__main__.py  :373] debug: True',
 '[2021-04-01 20:31:58,453][messages.py  :133] Progress bar: 1%',
 "[2021-04-01 20:31:58,916][context.py   :499] teams_parameters: ['/orbit/test-env-public/teams/lake-creator/context', '/orbit/test-env-public/teams/lake-creator/manifest', '/orbit/test-env-public/teams/lake-creator/team', '/orbit/test-env-public/teams/lake-creator/user/profiles', '/orbit/test-env-public/teams/lake-user/context', '/orbit/test-env-public/teams/lake-user/manifest', '/orbit/test-env-public/teams/lake-user/team', '/orbit/test-env-public/teams/lake-user/user/profiles']",
 '[ Info ] 

Lets get the image address from the output of the previous command

In [10]:
look_for = 'ECR Image Address='
image = None
for o in output:
    if look_for in o:
        image = o[o.index(look_for) + len(look_for):]
        print(image)

assert(image != None)       
    

495869084367.dkr.ecr.us-west-2.amazonaws.com/orbit-test-env-public-users-jupyter-with-apps


In [11]:
# check that the image was built
import json
print(repository_name)
images = !aws ecr list-images --repository-name $repository_name
images = "".join(images)
im = json.loads(images)
print(im['imageIds'])
assert(len(im['imageIds']) > 0)

orbit-test-env-public-users-jupyter-with-apps
[{'imageDigest': 'sha256:8b54baa722e25b8a1712b5daa09e3e1cde94db9ba58d54bbc7df3e401bd50030', 'imageTag': 'latest'}]


### Building the profile for the Image

In [12]:
import json
profile = {
        "display_name": 'Workbench Apps (mid-size)',
        "description": "Build an image with additional data and development applications",
        "kubespawner_override": {
            "image": image,
            "cpu_guarantee": 4,
            "cpu_limit": 4,
            "mem_guarantee": "4G",
            "mem_limit": "4G"
        }
}

with open("profile.json", 'w') as f:
    json.dump(profile, f)


In [13]:
!cat profile.json

{"display_name": "Workbench Apps (mid-size)", "description": "Build an image with additional data and development applications", "kubespawner_override": {"image": "495869084367.dkr.ecr.us-west-2.amazonaws.com/orbit-test-env-public-users-jupyter-with-apps", "cpu_guarantee": 4, "cpu_limit": 4, "mem_guarantee": "4G", "mem_limit": "4G"}}

In [14]:
!orbit build profile --debug --env $env_name --team lake-user profile.json

[2021-04-01 20:40:21,511][__main__.py  :481] env: test-env-public
[2021-04-01 20:40:21,511][__main__.py  :482] team: lake-user
[2021-04-01 20:40:21,511][__main__.py  :484] profile: {"display_name": "Workbench Apps (mid-size)", "description": "Build an image with additional data and development applications", "kubespawner_override": {"image": "495869084367.dkr.ecr.us-west-2.amazonaws.com/orbit-test-env-public-users-jupyter-with-apps", "cpu_guarantee": 4, "cpu_limit": 4, "mem_guarantee": "4G", "mem_limit": "4G"}}
[2021-04-01 20:40:21,511][__main__.py  :485] debug: True
[2021-04-01 20:40:21,511][messages.py  :133] Progress bar: 1%
[ Info ] Retrieving existing profiles
[2021-04-01 20:40:21,511][image.py     : 35] Trying to read profiles from SSM parameter (/orbit/test-env-public/teams/lake-user/user/profiles).
[2021-04-01 20:40:21,663][image.py     :102] Existing user profiles for team lake-user: [{'description': 'Use simple custom image', 'display_name': 'Workbench Apps (mid-size)', 'kube

## Users can now use the new profile to use new apps